In [1]:
import geopandas as gpd
import pandas as pd
import ee
import geemap
ee.Authenticate()
ee.Initialize(project='pacaas-novas')

In [2]:
pacaas_area = gpd.read_file('C:/Users/ppgar/Downloads/QGIS/Pacaas_novos/pacaas_novas.geojson')

In [3]:
pacaas_area = pacaas_area.geometry.iloc[0].__geo_interface__

In [4]:
pacaas_area = ee.Geometry(pacaas_area)
buffer = pacaas_area.buffer(5000)
buffer_diff = buffer.difference(pacaas_area)

In [5]:
def calc_ndvi(image):
    nir = image.select('B8')
    red = image.select('B4')
    ndvi = nir.subtract(red).divide(nir.add(red)).rename('NDVI')
    return image.addBands(ndvi)

In [6]:
def mask_clouds(image):
    cloud_mask = image.select('QA60').eq(0)
    return image.updateMask(cloud_mask)

satelite_2016 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-05-01', '2016-09-30').map(mask_clouds).median().clip(buffer).select(['B11', 'B8', 'B4'])
satelite_2016_buffer = ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-05-01', '2016-09-30').map(mask_clouds).median().clip(buffer_diff).select(['B11', 'B8', 'B4'])

In [7]:
pacaas_hist = pd.DataFrame()
for year in range(2015,2025):
    ndvi_mean = calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(pacaas_area)).select(['NDVI']).reduceRegion('mean')
    ndvi_buffer = calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(buffer_diff)).select(['NDVI']).reduceRegion('mean')
    aux = pd.DataFrame({'Ano': [year],
                        'Area_NDVI': [ndvi_mean.get('NDVI').getInfo()],
                        'Buffer_NDVI': [ndvi_buffer.get('NDVI').getInfo()],
                        'Area': [ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(pacaas_area)],
                        'Buffer': [ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate(f'{year}-06-01', f'{year}-09-30').map(mask_clouds).median().clip(buffer_diff)]
                        })
    pacaas_hist = pd.concat([pacaas_hist, aux])
pacaas_hist.reset_index(inplace = True)

In [8]:
ndvi_2024 = calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2024-06-01', '2024-08-30').map(mask_clouds).median().clip(buffer)).select(['NDVI'])
ndvi_2016 = calc_ndvi(ee.ImageCollection('COPERNICUS/S2_HARMONIZED').filterDate('2016-06-01', '2016-08-30').map(mask_clouds).median().clip(buffer)).select(['NDVI'])
ndvi_water = ndvi_2024.lt(0)
ndvi_pastures = ndvi_2024.lt(0.4).And(ndvi_2024.gt(0))
ndvi_forests = ndvi_2024.gte(0.4)
pastures_masked = ndvi_pastures.updateMask(ndvi_pastures.neq(0))
waterbodies_masked = ndvi_water.updateMask(ndvi_water.neq(0))
forests_masked = ndvi_forests.updateMask(ndvi_forests.neq(0))
map =  geemap.Map()
map.set_center(-65, -11.3, 9.5)
map.addLayer(pastures_masked, {'min': 0,'max': 1,'palette': ['white', 'white', 'red']}, opacity = 0.8, name = 'Low density vegetation')
map.addLayer(waterbodies_masked,{'min': 0,'max': 1,'palette': ['white', 'white', 'blue']}, opacity=0.8, name = 'Water bodies')
map.addLayer(forests_masked,{'min': 0,'max': 1,'palette': ['white', 'white', 'green']}, opacity = 0.8, name = 'Dense Forests')

map


Map(center=[-11.3, -65], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(…

In [10]:
palette = ['red', 'orange', 'yellow', 'lightgreen', 'darkgreen']
mapa = geemap.Map()
mapa.addLayer(ndvi_2024.subtract(ndvi_2016), {'min': -0.5, 'max': 0.5, 'palette': palette}, 'NDVI Difference')
mapa


Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…